# Identifying Fraud at Enron Using Emails and Financial Data

## Project Introduction

In 2000, Enron was one of the largest companies in the United States. By 2002, it had collapsed into bankruptcy due to widespread corporate fraud. In the resulting Federal investigation, there was a significant amount of typically confidential information entered into public record, including tens of thousands of emails and detailed financial data for top executives.
For this project, predictive models were built using scikit learn, numpy, and pandas modules in Python. The target of the predictions were persons-of-interest (POI’s) who were ‘individuals who were indicted, reached a settlement, or plea deal with the government, or testified in exchange for prosecution immunity.’ Financial compensation data and aggregate email statistics from the Enron Corpus were used as features for prediction.

The goal of this project is to build a prediction model to identify persons-of-interest (POI’s.)

**Importing Necessary Libraries**

In [30]:
import sys
import pickle
import numpy as np
import pandas as pd
import sklearn
import matplotlib as plt
%matplotlib inline

In [31]:
import pickle

enron_data = pickle.load(open("C:/Users/Geekquad/ud120-projects/final_project/final_project_dataset_modified_unix.pkl", "rb"))

## Understanding the Dataset

### Data Exploration 

Addresses the most important characteristics of the dataset and uses these characteristics to inform their analysis.

**Important characteristics include: **
-  Size of the Enron Dataset
-  Features in the Enron Dataset 
-  Finding POI's in the Enron Data
-  Query the Dataset - 1
- Query the Dataset 2
- Query the Dataset 3
- Follow the Money
- Dealing with Unfilled Features
- Missing POI's 1
- Missing POI's 2